In [1]:
# Import essential libraries
import tensorflow as tf
import numpy as np
import json
import pickle
import sys
import tensorflow.keras as keras
import pandas as pd
from matplotlib import pyplot as plt
from tensorflow.keras import backend as K
import tensorflow as tf
import os
from helper_functions import convert_ndarray
from custom_objects import custom_dead_relu_initializer

gpu = len(tf.config.list_physical_devices('GPU')) > 0
print("GPU is", "available" if gpu else "NOT AVAILABLE")

GPU is available


In [114]:
with open('saved_precalculations/' + "two_layer_mlp_net" + '/class_average_signals.pickle', 'rb') as handle:
    signals = pickle.load(handle)

In [115]:
with open('saved_precalculations/' + "two_layer_mlp_net" + '/class_average_gradients.pickle', 'rb') as handle:
    gradients = pickle.load(handle)

In [85]:
len(signals)

10

In [88]:
signals[0][0].shape

(784, 200)

In [116]:
gradients[3][2].shape

(100, 10)

In [117]:
gradient_weighted_sigs = [[signals[i][j] * gradients[i][j] for j in range(len(signals[i]))] for i in range(len(signals))]

In [118]:
gradient_weighted_sigs[0][1]

array([[ 7.05833259e-07,  6.52226106e-07, -5.25952237e-06, ...,
        -1.17322782e-06, -3.53070141e-06,  4.09671372e-08],
       [-2.14141255e-05, -7.76586440e-09, -3.07346090e-05, ...,
         2.60904199e-06, -1.24215840e-05, -5.00312717e-06],
       [ 1.48388252e-04,  6.67992499e-05,  1.67839335e-05, ...,
        -3.28727560e-06,  7.00173696e-05, -7.80813934e-06],
       ...,
       [ 4.68766217e-07,  3.64170955e-06,  3.52426753e-07, ...,
        -3.73427156e-07, -2.30238652e-06,  1.46519551e-06],
       [-5.16810871e-07, -8.87334863e-08, -4.26333827e-05, ...,
        -5.66259132e-07,  4.46968443e-05, -2.73269802e-06],
       [ 1.36928775e-05, -8.31473426e-06, -2.94134406e-06, ...,
        -3.49910442e-06,  3.80518418e-06,  2.27976056e-08]], dtype=float32)

In [3]:
model_name = 'two_layer_mlp_net'
model = tf.keras.models.load_model("saved_models/two_layer_mlp_net.keras")

2024-08-05 09:38:19.089801: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M2
2024-08-05 09:38:19.089840: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2024-08-05 09:38:19.089852: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
2024-08-05 09:38:19.090471: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:303] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-08-05 09:38:19.090819: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:269] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [71]:
batch , _ = next(iter(dataset))
weights = [var for var in model.trainable_variables if 'kernel' in var.name]

with tf.GradientTape() as tape:
    tape.watch(batch)
    predictions = model(batch)
    loss = model.loss(_, predictions)

# Calculate the gradient of the loss with respect to the input
gradients = tape.gradient(loss, weights)



In [54]:
len(batch)

32

In [72]:
#gradients = [gradients[i].numpy() for i in range(len(gradients))]
gradients2 = [gradients[i] + (gradients[i] * 32)for i in range(len(gradients))]

In [74]:
gradients[1][0]

<tf.Tensor: shape=(100,), dtype=float32, numpy=
array([ 4.22038247e-05,  1.89894618e-05, -1.06671951e-05, -7.96097703e-03,
       -2.20582569e-05,  3.80422316e-05, -2.35561654e-03,  2.52776372e-05,
        3.76275580e-07, -3.94119741e-03, -1.95086794e-03,  9.23395055e-06,
       -8.01648130e-05,  4.24244172e-06,  4.56956423e-06,  9.26500979e-06,
       -3.88511806e-04, -7.16006209e-04, -1.08230561e-05,  2.43899412e-03,
       -7.76512970e-05, -3.34875920e-04,  6.92457834e-05,  8.40063803e-05,
        3.01868859e-05, -3.03241145e-03,  1.83270149e-05,  3.75981131e-08,
       -2.92439427e-06, -1.13837143e-06,  1.30451517e-03,  2.19090446e-03,
        2.19556978e-06, -2.30834085e-05,  6.03580447e-05, -5.58350403e-05,
        1.61619482e-06,  1.51452166e-03, -2.08827496e-05, -1.58912546e-04,
       -5.94750419e-03, -1.91155457e-04, -5.85558919e-05, -6.01988759e-05,
        2.50768964e-04,  7.61557749e-05, -1.86728593e-03, -3.42256544e-06,
       -5.05291723e-07, -1.77942403e-03,  1.31747220

In [73]:
gradients2[1][0]

<tf.Tensor: shape=(100,), dtype=float32, numpy=
array([ 1.39272620e-03,  6.26652210e-04, -3.52017436e-04, -2.62712240e-01,
       -7.27922481e-04,  1.25539361e-03, -7.77353495e-02,  8.34161998e-04,
        1.24170938e-05, -1.30059510e-01, -6.43786415e-02,  3.04720365e-04,
       -2.64543877e-03,  1.40000571e-04,  1.50795619e-04,  3.05745314e-04,
       -1.28208892e-02, -2.36282051e-02, -3.57160839e-04,  8.04868042e-02,
       -2.56249285e-03, -1.10509051e-02,  2.28511076e-03,  2.77221063e-03,
        9.96167189e-04, -1.00069575e-01,  6.04791509e-04,  1.24073767e-06,
       -9.65050131e-05, -3.75662567e-05,  4.30490002e-02,  7.22998455e-02,
        7.24538040e-05, -7.61752483e-04,  1.99181540e-03, -1.84255629e-03,
        5.33344282e-05,  4.99792136e-02, -6.89130742e-04, -5.24411397e-03,
       -1.96267635e-01, -6.30812999e-03, -1.93234440e-03, -1.98656297e-03,
        8.27537570e-03,  2.51314067e-03, -6.16204366e-02, -1.12944661e-04,
       -1.66746267e-05, -5.87209910e-02,  4.34765825

In [52]:
gradients[3]

IndexError: list index out of range

In [13]:
model.loss(_, layer_outputs)

<tf.Tensor: shape=(), dtype=float32, numpy=0.0010066299>

In [10]:
layer_gradients.shape

AttributeError: 'NoneType' object has no attribute 'shape'

In [52]:
reshaped_results = [raw_data[i].reshape(len(raw_data[i]),
                                                                      len(raw_data[i][0]) * len(
                                                                          raw_data[i][0][0])) for i in
                                     range(len(raw_data))]
            # Cut off input layer neurons
output_data = reshaped_results[1:]

reshaped_results = [raw_data[i].reshape(len(raw_data[i][0]),
                                                                      len(raw_data[i]) * len(
                                                                          raw_data[i][0][0])) for i in
                                     range(len(raw_data))]
            # Cut off output layer neurons
input_data = reshaped_results[:-1]
layerwise_neuron_data = [np.concatenate((output_data[i], input_data[i]), axis=1) for i in range(len(output_data))]

In [60]:
output_data[0].shape



In [2]:
K.gradients()

TypeError: gradients() missing 2 required positional arguments: 'loss' and 'variables'

In [63]:
input_data[1].shape

(100, 115200)

In [62]:
layerwise_neuron_data[1].shape

(100, 120960)

In [45]:
layerwise_neuron_data=layerwise_neuron_data[1:]

In [48]:
layerwise_neuron_data[0].shape

(200, 57600)

In [4]:
(x_train_unnormalized, y_train), (x_test_unnormalized, y_test) = keras.datasets.mnist.load_data()
x_train = x_train_unnormalized / 255.0
x_test = x_test_unnormalized / 255.0
# Inserting error:
#y_train = np.where(y_train == 7, 1, y_train)
dataset = (tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(64)).batch(32)

In [3]:
# Ideas for inserting errors:
# Manipulate Class so that it results in error
# Cause Exploding / Vanishing Gradients.
# Look at adverserial attacks
# Look at Dropout

In [8]:
# Vanishing gradient model
model = keras.Sequential()
model.add(keras.Input(shape=(28,28,)))
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(200, activation='tanh',  kernel_regularizer=keras.regularizers.l1(l=0.01)))
model.add(keras.layers.Dense(100, activation='tanh',  kernel_regularizer=keras.regularizers.l1(l=0.01)))
model.add(keras.layers.Dense(10))


# Apply loss function and optimizer, then compile the model
loss_fn = keras.losses.SparseCategoricalCrossentropy(from_logits=True)
optimizer = keras.optimizers.Adam()
model.compile(optimizer=optimizer, loss=loss_fn, metrics=["acc"])

# Train the model on dataset for given epochs
model.fit(x_train, y_train, batch_size=32, epochs=10, validation_data=(x_test, y_test))

Epoch 1/10
1875/1875 [==============================] - 3s 1ms/step - loss: 2.9755 - acc: 0.7831 - val_loss: 1.2292 - val_acc: 0.8545
Epoch 2/10
1875/1875 [==============================] - 3s 1ms/step - loss: 1.1887 - acc: 0.8541 - val_loss: 1.1331 - val_acc: 0.8686
Epoch 3/10
1875/1875 [==============================] - 3s 1ms/step - loss: 1.1204 - acc: 0.8655 - val_loss: 1.0653 - val_acc: 0.8677
Epoch 4/10
1875/1875 [==============================] - 3s 1ms/step - loss: 1.0811 - acc: 0.8688 - val_loss: 1.0780 - val_acc: 0.8611
Epoch 5/10
1875/1875 [==============================] - 2s 1ms/step - loss: 1.0543 - acc: 0.8704 - val_loss: 1.0124 - val_acc: 0.8772
Epoch 6/10
1875/1875 [==============================] - 3s 1ms/step - loss: 1.0311 - acc: 0.8740 - val_loss: 1.0023 - val_acc: 0.8705
Epoch 7/10
1875/1875 [==============================] - 3s 1ms/step - loss: 1.0168 - acc: 0.8730 - val_loss: 1.0174 - val_acc: 0.8774
Epoch 8/10
1875/1875 [==============================] - 3s 1ms

In [17]:
# Exploading gradient model
model = keras.Sequential()
model.add(keras.Input(shape=(28,28,)))
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(200, activation='relu'))
model.add(keras.layers.Dense(100, activation='relu'))
model.add(keras.layers.Dense(10))


# Apply loss function and optimizer, then compile the model
loss_fn = keras.losses.SparseCategoricalCrossentropy(from_logits=True)
optimizer = keras.optimizers.SGD(learning_rate=1.0)
model.compile(optimizer=optimizer, loss=loss_fn, metrics=["acc"])

# Train the model on dataset for given epochs
model.fit(x_train, y_train, batch_size=32, epochs=1, validation_data=(x_test, y_test))

Epoch 1/10
1875/1875 [==============================] - 3s 1ms/step - loss: 1.4224 - acc: 0.5134 - val_loss: 0.9988 - val_acc: 0.6633
Epoch 2/10
1875/1875 [==============================] - 2s 1ms/step - loss: 1.1662 - acc: 0.5772 - val_loss: 1.7400 - val_acc: 0.5169
Epoch 3/10
1875/1875 [==============================] - 2s 1ms/step - loss: 1.5438 - acc: 0.3706 - val_loss: 1.4963 - val_acc: 0.3680
Epoch 4/10
1875/1875 [==============================] - 2s 1ms/step - loss: 1.7170 - acc: 0.3090 - val_loss: 1.7053 - val_acc: 0.2816
Epoch 5/10
1875/1875 [==============================] - 2s 1ms/step - loss: 1.6202 - acc: 0.2989 - val_loss: 1.5796 - val_acc: 0.2878
Epoch 6/10
1875/1875 [==============================] - 2s 1ms/step - loss: 1.6975 - acc: 0.2867 - val_loss: 1.6433 - val_acc: 0.3034
Epoch 7/10
1875/1875 [==============================] - 2s 1ms/step - loss: 1.9821 - acc: 0.2548 - val_loss: 1.8137 - val_acc: 0.2460
Epoch 8/10
1875/1875 [==============================] - 2s 1ms

In [20]:
# Overfitted Model
model = keras.Sequential()
model.add(keras.Input(shape=(28,28,)))
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(200, activation='tanh'))
model.add(keras.layers.Dense(100, activation='tanh'))
model.add(keras.layers.Dense(10))


# Apply loss function and optimizer, then compile the model
loss_fn = keras.losses.SparseCategoricalCrossentropy(from_logits=True)
optimizer = keras.optimizers.Adam()
model.compile(optimizer=optimizer, loss=loss_fn, metrics=["acc"])

# Train the model on dataset for given epochs
model.fit(x_train, y_train, batch_size=32, epochs=200, validation_data=(x_test, y_test))

Epoch 1/200
1875/1875 [==============================] - 3s 1ms/step - loss: 0.2440 - acc: 0.9273 - val_loss: 0.1359 - val_acc: 0.9587
Epoch 2/200
1875/1875 [==============================] - 2s 1ms/step - loss: 0.1141 - acc: 0.9656 - val_loss: 0.0968 - val_acc: 0.9706
Epoch 3/200
1875/1875 [==============================] - 2s 1ms/step - loss: 0.0776 - acc: 0.9762 - val_loss: 0.0899 - val_acc: 0.9717
Epoch 4/200
1875/1875 [==============================] - 2s 1ms/step - loss: 0.0559 - acc: 0.9827 - val_loss: 0.0854 - val_acc: 0.9732
Epoch 5/200
1875/1875 [==============================] - 2s 1ms/step - loss: 0.0419 - acc: 0.9865 - val_loss: 0.0845 - val_acc: 0.9723
Epoch 6/200
1875/1875 [==============================] - 2s 1ms/step - loss: 0.0325 - acc: 0.9899 - val_loss: 0.0780 - val_acc: 0.9777
Epoch 7/200
1875/1875 [==============================] - 3s 2ms/step - loss: 0.0254 - acc: 0.9916 - val_loss: 0.0769 - val_acc: 0.9752
Epoch 8/200
1875/1875 [==============================] 

In [33]:
dataset_size = len(x_train)
indices = np.random.choice(dataset_size, dataset_size//20, replace=False)

x_train_subset = x_train[indices]
y_train_subset = y_train[indices]

# Overfitted Model
model = keras.Sequential()
model.add(keras.Input(shape=(28,28,)))
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(200, activation='tanh'))
model.add(keras.layers.Dense(100, activation='tanh'))
model.add(keras.layers.Dense(10))


# Apply loss function and optimizer, then compile the model
loss_fn = keras.losses.SparseCategoricalCrossentropy(from_logits=True)
optimizer = keras.optimizers.Adam()
model.compile(optimizer=optimizer, loss=loss_fn, metrics=["acc"])

# Train the model on dataset for given epochs
model.fit(x_train_subset, y_train_subset, batch_size=32, epochs=1, validation_data=(x_test, y_test))

94/94 [==============================] - 1s 7ms/step - loss: 0.7113 - acc: 0.7930 - val_loss: 0.4138 - val_acc: 0.8770


In [3]:
# Untrained model, to compare to trained ones.
model = keras.Sequential()
model.add(keras.Input(shape=(28,28,)))
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(200, activation='tanh'))
model.add(keras.layers.Dense(100, activation='tanh'))
model.add(keras.layers.Dense(10))


# Apply loss function and optimizer, then compile the model
loss_fn = keras.losses.SparseCategoricalCrossentropy(from_logits=True)
optimizer = keras.optimizers.Adam()
model.compile(optimizer=optimizer, loss=loss_fn, metrics=["acc"])

In [26]:
# Train the model on dataset for given epochs
model.fit(x_train, y_train, batch_size=32, epochs=10, validation_data=(x_test, y_test))

Epoch 1/10
1875/1875 [==============================] - 3s 1ms/step - loss: 0.8893 - acc: 0.8148 - val_loss: 0.3177 - val_acc: 0.9103
Epoch 2/10
1875/1875 [==============================] - 3s 1ms/step - loss: 0.3112 - acc: 0.9117 - val_loss: 0.2706 - val_acc: 0.9221
Epoch 3/10
1875/1875 [==============================] - 3s 1ms/step - loss: 0.2071 - acc: 0.9400 - val_loss: 0.1602 - val_acc: 0.9517
Epoch 4/10
1875/1875 [==============================] - 3s 1ms/step - loss: 0.1361 - acc: 0.9595 - val_loss: 0.1239 - val_acc: 0.9623
Epoch 5/10
1875/1875 [==============================] - 3s 1ms/step - loss: 0.0999 - acc: 0.9694 - val_loss: 0.1406 - val_acc: 0.9582
Epoch 6/10
1875/1875 [==============================] - 3s 1ms/step - loss: 0.0794 - acc: 0.9755 - val_loss: 0.0963 - val_acc: 0.9701
Epoch 7/10
1875/1875 [==============================] - 3s 1ms/step - loss: 0.0625 - acc: 0.9803 - val_loss: 0.0911 - val_acc: 0.9737
Epoch 8/10
1875/1875 [==============================] - 3s 1ms

In [34]:
#first_layer_weights = model.layers[3].get_weights()[0]
#dead_neurons = np.sum(np.all(first_layer_weights <= 0, axis=0))
#print(f'Anzahl der "toten" Neuronen: {dead_neurons}')

Anzahl der "toten" Neuronen: 0


In [4]:
# Save the model at the correct path
name = "two_layer_mlp_untrained"
file_name = name + ".keras"
model.save('saved_models/' + file_name)

In [8]:
create_precalculations(name)

Starting with calculations
Currently at batch: 0
Currently at batch: 1
Currently at batch: 2
Currently at batch: 3
Currently at batch: 4
Currently at batch: 5
Currently at batch: 6
Currently at batch: 7
Currently at batch: 8
Currently at batch: 9
Currently at batch: 10
Currently at batch: 11
Currently at batch: 12
Currently at batch: 13
Currently at batch: 14
Currently at batch: 15
Currently at batch: 16
Currently at batch: 17
Done with calculations. Saving to file.
Starting with calculations
Currently at batch: 0
Currently at batch: 1
Currently at batch: 2
Currently at batch: 3
Currently at batch: 4
Currently at batch: 5
Currently at batch: 6
Currently at batch: 7
Currently at batch: 8
Currently at batch: 9
Currently at batch: 10
Currently at batch: 11
Currently at batch: 12
Currently at batch: 13
Currently at batch: 14
Currently at batch: 15
Currently at batch: 16
Currently at batch: 17
Done with calculations. Saving to file.
Starting with calculations
Currently at batch: 0
Currently

In [113]:
model_name = 'two_layer_mlp_net'
model = tf.keras.models.load_model("saved_models/two_layer_mlp_net.keras")
create_precalculations(model_name)

[Errno 17] File exists: 'saved_precalculations/two_layer_mlp_net'
Starting with calculations
Currently at batch: 0
Currently at batch: 1
Currently at batch: 2
Currently at batch: 3
Currently at batch: 4
Currently at batch: 5
Currently at batch: 6
Currently at batch: 7
Currently at batch: 8
Currently at batch: 9
Currently at batch: 10
Currently at batch: 11
Currently at batch: 12
Currently at batch: 13
Currently at batch: 14
Currently at batch: 15
Currently at batch: 16
Currently at batch: 17
Done with calculations. Saving to file.
Starting with calculations
Currently at batch: 0
Currently at batch: 1
Currently at batch: 2
Currently at batch: 3
Currently at batch: 4
Currently at batch: 5
Currently at batch: 6
Currently at batch: 7
Currently at batch: 8
Currently at batch: 9
Currently at batch: 10
Currently at batch: 11
Currently at batch: 12
Currently at batch: 13
Currently at batch: 14
Currently at batch: 15
Currently at batch: 16
Currently at batch: 17
Done with calculations. Saving t

2024-08-05 14:21:30.442402: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-08-05 14:21:42.859875: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


Done with class 0..
Done with class 1..
Done with class 2..
Done with class 3..
Done with class 4..
Done with class 5..
Done with class 6..
Done with class 7..
Done with class 8..
Done with class 9..
Computing overall activation averages..
Averaging activation classes..
Averaging gradient classes..
Averaging correct prediction activation classes..
Averaging incorrect prediction activation classes..
Computing overall signal averages..
Averaging signal classes..
Averaging correct prediction signal classes..
Averaging incorrect prediction signal classes..
Saving results..
Done


In [77]:
# Add functions that need to be recomputed after every new model (precalculations)
# Add automatic creation of folder
def create_precalculations(model_name):
    model = keras.models.load_model('saved_models/' + model_name + '.keras')

    # Create Folder for precalculations, if it doesn't exist yet.
    save_file_path = 'saved_precalculations/' + model_name
    try:
        os.mkdir(save_file_path)
    except OSError as error:
        print(error)

    create_subset(model, 'saved_precalculations/' + model_name + '/subset_activations_1.pickle')
    create_subset(model, 'saved_precalculations/' + model_name + '/subset_activations_2.pickle')
    create_subset(model, 'saved_precalculations/' + model_name + '/subset_activations_3.pickle')

    calc_class_average_activation_and_signals(model, model_name)

    print("Done")

In [79]:
# TODO: Shift these functions to a seperate .py file and import them at the beginning of the nb
def create_subset(model, file_path=None):
    dataset = (tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(64)).batch(32)

    dataset_size = sum(1 for _ in dataset)
    one_percent_size = dataset_size // 100

    # Take the first 1% of the dataset
    one_percent_dataset = dataset.take(one_percent_size)

    functions = []
    results = []
    for i in range(len(model.layers)):
        functions.append(K.function([model.layers[i].input],[model.layers[i].output]))

    print("Starting with calculations")
    for i, (batch, y) in enumerate(one_percent_dataset):
        print("Currently at batch: " + str(i))

        output = functions[0](batch)
        if i == 0:
            results.append(np.transpose(np.squeeze(output, axis=0)))
        else:
            results[0] = np.concatenate([results[0], np.transpose(np.squeeze(output, axis=0))], axis=1)

        for j in range(1, len(functions)):
            tmp = functions[j](output)
            output = tmp
            if i == 0:
                results.append(np.transpose(np.squeeze(output, axis=0)))
            else:
                results[j] = np.concatenate([results[j], np.transpose(np.squeeze(output, axis=0))], axis=1)

    print("Done with calculations. Saving to file.")

    with open(file_path, 'wb') as handle:
        pickle.dump(results, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [76]:
def create_subset_signals(model, file_path=None):
    dataset = (tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(64)).batch(32)

    dataset_size = sum(1 for _ in dataset)
    one_percent_size = dataset_size // 100

    # Take the first 1% of the dataset
    one_percent_dataset = dataset.take(one_percent_size)

    functions = []
    transposed_weights = []
    results = []
    for i in range(len(model.layers)):
        functions.append(K.function([model.layers[i].input],model.layers[i].output))
        if len(model.layers[i].get_weights()) == 0:
            transposed_weights.append([])
        else:
            transposed_weights.append(tf.transpose(model.layers[i].get_weights()[0]))

    print("Starting with signal calculations")
    for i, (batch, y) in enumerate(one_percent_dataset):
        print("Currently at batch: " + str(i))

        output = functions[0](batch)

        for j in range(1, len(functions)):
            output = functions[j](output)
            if i == 0:
                results.append(tf.expand_dims((tf.einsum('bi,ij->bji', output, transposed_weights[j])), axis=0))
            else:
                results[j-1] = np.concatenate([results[j-1], tf.expand_dims(tf.einsum('bi,ij->bji', output, transposed_weights[j]),axis=0)], axis=0)

    print("Done with signal calculations. Saving to file.")
    # Shapes results to (input, output, batch_size * amount_of_batches) shape, as we need the 2D array for UMAP and ClusterTrees
    reshaped_array = [results[i].reshape(len(results[i][0][0]), len(results[i][0][0][0]), len(results[i]) * len(results[i][0])) for i in range(len(results))]

    with open(file_path, 'wb') as handle:
        pickle.dump(reshaped_array, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [10]:
def calc_average_activation_and_signals(model, file_path_activation, file_path_signals):
    dataset = (tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(64)).batch(32, drop_remainder=True)

    functions = []
    transposed_weights = []
    for i in range(len(model.layers)):
        functions.append(K.function([model.layers[i].input],model.layers[i].output))
        if len(model.layers[i].get_weights()) == 0:
            transposed_weights.append([])
        else:
            transposed_weights.append(tf.transpose(model.layers[i].get_weights()[0]))

    average_signals_per_layer = []
    average_activations_per_layer = []
    print("Starting with calculations")
    for i, (batch, y) in enumerate(dataset):
        print("Currently at batch: " + str(i))

        output = functions[0](batch)
        if i == 0:
            average_activations_per_layer.append(tf.expand_dims(tf.reduce_mean(output, axis=0), axis=0))
        else:
            average_activations_per_layer[0] = np.concatenate([average_activations_per_layer[0], tf.expand_dims(tf.reduce_mean(output, axis=0), axis=0)], axis=0)

        for j in range(1, len(functions)):
            output = functions[j](output)
            if i == 0:
                average_activations_per_layer.append(tf.expand_dims(tf.reduce_mean(output, axis=0), axis=0))
                average_signals_per_layer.append(tf.expand_dims(tf.reduce_mean(tf.einsum('bi,ij->bji', output, transposed_weights[j]),axis=0), axis=0))
            else:
                average_activations_per_layer[j] = np.concatenate([average_activations_per_layer[j], tf.expand_dims(tf.reduce_mean(output, axis=0), axis=0)], axis=0)
                average_signals_per_layer[j-1] = np.concatenate([average_signals_per_layer[j-1], tf.expand_dims(tf.reduce_mean(tf.einsum('bi,ij->bji', output, transposed_weights[j]),axis=0), axis=0)],axis=0)

    print("Done with calculations, now averaging results..")

    for index in range(len(average_activations_per_layer)):
        average_activations_per_layer[index] = tf.reduce_mean(average_activations_per_layer[index], axis=0).numpy()

    for index in range(len(average_signals_per_layer)):
        average_signals_per_layer[index] = tf.reduce_mean(average_signals_per_layer[index], axis=0).numpy()

    print("Done with calculations. Saving to file.")

    with open(file_path_activation, 'wb') as handle:
        pickle.dump(average_activations_per_layer, handle, protocol=pickle.HIGHEST_PROTOCOL)

    with open(file_path_signals, 'wb') as handle:
        pickle.dump(average_signals_per_layer, handle, protocol=pickle.HIGHEST_PROTOCOL)

    return

In [12]:
def calc_average_signal(model, file_path):
    dataset = (tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(64)).batch(32)
    summed_signal_per_layer = []

    print("Starting with calculations")
    for i, (batch, _) in enumerate(dataset):
        print("Currently at batch: " + str(i))
        inputs = batch
        for j in range(len(model.layers)-1):
            # Compute the activations of the current layer
            activations = model.layers[j](inputs)
            # Get the weights and biases of the current layer
            weights, biases = model.layers[j+1].get_weights()
            # Compute the outgoing signals (weight times activation)
            expanded_activations = np.expand_dims(activations, axis=-1)
            expanded_weights = np.expand_dims(weights, axis=0)
            outgoing_signals = expanded_activations * expanded_weights
            # Store the outgoing signals for the current layer
            if i == 0:
                summed_signal_per_layer.append(tf.reduce_sum(outgoing_signals, axis=0))
            else:
                summed_signal_per_layer[j] += tf.reduce_sum(outgoing_signals, axis=0)
            # Update inputs for the next layer
            inputs = activations


    print("Done with calculations, averaging results..")
    average_signal_per_layer = []
    for matrix in summed_signal_per_layer:
        average_signal_per_layer.append((matrix / x_train.shape[0]).numpy())
    with open(file_path, 'wb') as handle:
        pickle.dump(average_signal_per_layer, handle, protocol=pickle.HIGHEST_PROTOCOL)
    return

In [11]:
model_name = 'two_layer_mlp_confused_class'
model = keras.models.load_model("saved_models/" + model_name + ".keras")

calc_class_average_activation_and_signals(model, model_name)

Done with class 0..
Done with class 1..
Done with class 2..
Done with class 3..
Done with class 4..
Done with class 5..
Done with class 6..
Done with class 7..
Done with class 8..
Done with class 9..
Computing overall activation averages..
Averaging activation classes..
Averaging correct prediction activation classes..
Averaging incorrect prediction activation classes..
Computing overall signal averages..
Averaging signal classes..
Averaging correct prediction signal classes..
Averaging incorrect prediction signal classes..
Saving results..


In [112]:
def calc_class_average_activation_and_signals(model, model_name):
    (x_train_unnormalized, y_train), (x_test_unnormalized, y_test) = keras.datasets.mnist.load_data()
    x_train = x_train_unnormalized / 255.0
    x_test = x_test_unnormalized / 255.0
    batch_size = 32

    functions = []
    weights = []

    class_average_signals_per_layer = []
    class_average_activations_per_layer = []
    class_average_gradients_per_layer = []
    class_correct_average_signals_per_layer = []
    class_incorrect_average_signals_per_layer = []
    class_correct_average_activations_per_layer = []
    class_incorrect_average_activations_per_layer = []
    class_correct_amounts = []
    class_incorrect_amounts = []

    for i in range(len(model.layers)):
        functions.append(K.function([model.layers[i].input],model.layers[i].output))
        if len(model.layers[i].get_weights()) > 0:
            weights.append(model.layers[i].get_weights()[0])

    # Needed to only get gradients of weights and not biases
    weights = [var for var in model.trainable_variables if 'kernel' in var.name]

    # 10 is hardcoded for the amount of MNIST classes. In case there is a different model we need to change this part (and the dataloading of course)
    for class_index in range (10):
        class_images = x_train[y_train==class_index]
        class_labels = y_train[y_train==class_index]
        dataset = tf.data.Dataset.from_tensor_slices((class_images, class_labels)).batch(batch_size)
        class_activations = []
        class_signals = []
        class_gradients = []
        class_correct_average_activations = []
        class_incorrect_average_activations = []
        class_correct_average_signals = []
        class_incorrect_average_signals = []

        for i, (batch, labels) in enumerate(dataset):
            # verbose 0 for silent mode
            predictions = model.predict(batch, verbose = 0)
            prediction_correctness = np.argmax(predictions, axis=-1) == labels
            correct_predictions = np.sum(prediction_correctness)
            incorrect_predictions = np.sum(~prediction_correctness)
            correct_indices = np.where(prediction_correctness)[0]
            incorrect_indices = np.where(~prediction_correctness)[0]

            with tf.GradientTape() as tape:
                tape.watch(batch)
                predictions = model(batch)
                loss = model.loss(labels, predictions)

            # Calculate the gradient of the loss with respect to the input
            tmp_gradients = tape.gradient(loss, weights)

            output = functions[0](batch)
            if i == 0:
                class_activations.append(tf.expand_dims(tf.reduce_sum(output, axis=0), axis=0))
                sigs = tf.einsum('bi,ij->bij', output, weights[0])
                class_signals.append(tf.expand_dims(tf.reduce_sum(sigs, axis=0), axis=0))
                class_correct_amounts.append(correct_predictions)
                class_incorrect_amounts.append(incorrect_predictions)
                if correct_predictions > 0:
                    class_correct_average_activations.append(tf.expand_dims(tf.reduce_sum(output[correct_indices], axis=0), axis=0))
                    class_correct_average_signals.append(tf.expand_dims(tf.reduce_sum(tf.gather(sigs, tf.convert_to_tensor(correct_indices, dtype=tf.int64), axis=0), axis=0), axis=0))
                if incorrect_predictions > 0:
                    class_incorrect_average_activations.append(tf.expand_dims(tf.reduce_sum(output[incorrect_indices], axis=0), axis=0))
                    class_incorrect_average_signals.append(tf.expand_dims(tf.reduce_sum(tf.gather(sigs, tf.convert_to_tensor(incorrect_indices, dtype=tf.int64), axis=0), axis=0), axis=0))

                class_gradients = [tmp_gradients[i] * len(batch) for i in range(len(tmp_gradients))]

            else:
                class_activations[0] = tf.add(class_activations[0], tf.expand_dims(tf.reduce_sum(output, axis=0), axis=0))
                sigs = tf.einsum('bi,ij->bij', output, weights[0])
                class_signals[0] = tf.add(class_signals[0], tf.expand_dims(tf.reduce_sum(tf.einsum('bi,ij->bij', output, weights[0]),axis=0), axis=0))
                class_correct_amounts[class_index] += correct_predictions
                class_incorrect_amounts[class_index] += incorrect_predictions
                if correct_predictions > 0:
                    if len(class_correct_average_activations) == 0:
                        class_correct_average_activations.append(tf.expand_dims(tf.reduce_sum(output[correct_indices], axis=0), axis=0))
                        class_correct_average_signals.append(tf.expand_dims(tf.reduce_sum(tf.gather(sigs, tf.convert_to_tensor(correct_indices, dtype=tf.int64), axis=0), axis=0), axis=0))
                    else:
                        class_correct_average_activations[0] = tf.add(class_correct_average_activations[0], tf.expand_dims(tf.reduce_sum(output[correct_indices], axis=0), axis=0))
                        class_correct_average_signals[0] = tf.add(class_correct_average_signals[0], tf.expand_dims(tf.reduce_sum(tf.gather(sigs, tf.convert_to_tensor(correct_indices, dtype=tf.int64), axis=0), axis=0), axis=0))
                if incorrect_predictions > 0:
                    if len(class_incorrect_average_activations) == 0:
                        class_incorrect_average_activations.append(tf.expand_dims(tf.reduce_sum(output[incorrect_indices], axis=0), axis=0))
                        class_incorrect_average_signals.append(tf.expand_dims(tf.reduce_sum(tf.gather(sigs, tf.convert_to_tensor(incorrect_indices, dtype=tf.int64), axis=0), axis=0), axis=0))
                    else:
                        class_incorrect_average_activations[0] = tf.add(class_incorrect_average_activations[0], tf.expand_dims(tf.reduce_sum(output[incorrect_indices], axis=0), axis=0))
                        class_incorrect_average_signals[0] = tf.add(class_incorrect_average_signals[0], tf.expand_dims(tf.reduce_sum(tf.gather(sigs, tf.convert_to_tensor(incorrect_indices, dtype=tf.int64), axis=0), axis=0), axis=0))

                class_gradients = [class_gradients[i] + (tmp_gradients[i] * len(batch)) for i in range(len(tmp_gradients))]

            for j in range(1, len(functions)-1):
                output = functions[j](output)
                if i == 0:
                    class_activations.append(tf.expand_dims(tf.reduce_sum(output, axis=0), axis=0))
                    sigs = tf.einsum('bi,ij->bij', output, weights[j])
                    class_signals.append(tf.expand_dims(tf.reduce_sum(sigs, axis=0), axis=0))
                    if correct_predictions > 0:
                        class_correct_average_activations.append(tf.expand_dims(tf.reduce_sum(output[correct_indices], axis=0), axis=0))
                        class_correct_average_signals.append(tf.expand_dims(tf.reduce_sum(tf.gather(sigs, tf.convert_to_tensor(correct_indices, dtype=tf.int64), axis=0), axis=0), axis=0))
                    if incorrect_predictions > 0:
                        class_incorrect_average_activations.append(tf.expand_dims(tf.reduce_sum(output[incorrect_indices], axis=0), axis=0))
                        class_incorrect_average_signals.append(tf.expand_dims(tf.reduce_sum(tf.gather(sigs, tf.convert_to_tensor(incorrect_indices, dtype=tf.int64), axis=0), axis=0), axis=0))
                else:
                    class_activations[j] = tf.add(class_activations[j], tf.expand_dims(tf.reduce_sum(output, axis=0), axis=0))
                    sigs = tf.einsum('bi,ij->bij', output, weights[j])
                    class_signals[j] = tf.add(class_signals[j], tf.expand_dims(tf.reduce_sum(sigs, axis=0), axis=0))
                if correct_predictions > 0:
                    if len(class_correct_average_activations) <= j:
                        class_correct_average_activations.append(tf.expand_dims(tf.reduce_sum(output[correct_indices], axis=0), axis=0))
                        class_correct_average_signals.append(tf.expand_dims(tf.reduce_sum(tf.gather(sigs, tf.convert_to_tensor(correct_indices, dtype=tf.int64), axis=0), axis=0), axis=0))
                    else:
                        class_correct_average_activations[j] = tf.add(class_correct_average_activations[j], tf.expand_dims(tf.reduce_sum(output[correct_indices], axis=0), axis=0))
                        class_correct_average_signals[j] = tf.add(class_correct_average_signals[j], tf.expand_dims(tf.reduce_sum(tf.gather(sigs, tf.convert_to_tensor(correct_indices, dtype=tf.int64), axis=0), axis=0), axis=0))
                if incorrect_predictions > 0:
                    if len(class_incorrect_average_activations) <= j:
                        class_incorrect_average_activations.append(tf.expand_dims(tf.reduce_sum(output[incorrect_indices], axis=0), axis=0))
                        class_incorrect_average_signals.append(tf.expand_dims(tf.reduce_sum(tf.gather(sigs, tf.convert_to_tensor(incorrect_indices, dtype=tf.int64), axis=0), axis=0), axis=0))
                    else:
                        class_incorrect_average_activations[j] = tf.add(class_incorrect_average_activations[j], tf.expand_dims(tf.reduce_sum(output[incorrect_indices], axis=0), axis=0))
                        class_incorrect_average_signals[j] = tf.add(class_incorrect_average_signals[j], tf.expand_dims(tf.reduce_sum(tf.gather(sigs, tf.convert_to_tensor(incorrect_indices, dtype=tf.int64), axis=0), axis=0), axis=0))

            output = functions[len(functions)-1](output)
            if i == 0:
                class_activations.append(tf.expand_dims(tf.reduce_sum(output, axis=0), axis=0))
                if correct_predictions > 0:
                    class_correct_average_activations.append(tf.expand_dims(tf.reduce_sum(output[correct_indices], axis=0), axis=0))
                if incorrect_predictions > 0:
                    class_incorrect_average_activations.append(tf.expand_dims(tf.reduce_sum(output[incorrect_indices], axis=0), axis=0))
            else:
                class_activations[len(functions)-1] = tf.add(class_activations[len(functions)-1], tf.expand_dims(tf.reduce_sum(output, axis=0), axis=0))
                if correct_predictions > 0:
                    if len(class_correct_average_activations) <= len(functions)-1:
                        class_correct_average_activations.append(tf.expand_dims(tf.reduce_sum(output[correct_indices], axis=0), axis=0))
                    else:
                        class_correct_average_activations[len(functions)-1] = tf.add(class_correct_average_activations[len(functions)-1], tf.expand_dims(tf.reduce_sum(output[correct_indices], axis=0), axis=0))
                if incorrect_predictions > 0:
                    if len(class_incorrect_average_activations) <= len(functions)-1:
                        class_incorrect_average_activations.append(tf.expand_dims(tf.reduce_sum(output[incorrect_indices], axis=0), axis=0))
                    else:
                        class_incorrect_average_activations[len(functions)-1] = tf.add(class_incorrect_average_activations[len(functions)-1], tf.expand_dims(tf.reduce_sum(output[incorrect_indices], axis=0), axis=0))
            weights = [var for var in model.trainable_variables if 'kernel' in var.name]


        print("Done with class {}..".format(str(class_index)))

        class_average_activations_per_layer.append(class_activations)
        class_average_signals_per_layer.append(class_signals)
        class_average_gradients_per_layer.append(class_gradients)
        class_correct_average_activations_per_layer.append(class_correct_average_activations)
        class_incorrect_average_activations_per_layer.append(class_incorrect_average_activations)
        class_correct_average_signals_per_layer.append(class_correct_average_signals)
        class_incorrect_average_signals_per_layer.append(class_incorrect_average_signals)

    print("Computing overall activation averages..")
    average_activations_per_layer = []
    num_layers = len(class_average_activations_per_layer[0])
    for layer_index in range(num_layers):
        # Initialisieren Sie einen Tensor, um die summierten Aktivierungen für die aktuelle Schicht zu speichern
        summed_activations = tf.zeros_like(class_average_activations_per_layer[0][layer_index])

        # Addieren Sie die durchschnittlichen Aktivierungen der aktuellen Schicht über alle Klassen hinweg
        for class_activations in class_average_activations_per_layer:
            summed_activations = tf.add(summed_activations, class_activations[layer_index])

        # Fügen Sie den Durchschnitt der aktuellen Schicht zur Ergebnisliste hinzu
        average_activations_per_layer.append(summed_activations)

    for layer_index in range(num_layers):
        average_activations_per_layer[layer_index] = np.squeeze(tf.divide(average_activations_per_layer[layer_index], x_train.shape[0]).numpy())

    print("Averaging activation classes..")
    for class_index, class_activations in enumerate(class_average_activations_per_layer):
        for layer_index, layer in enumerate(class_activations):
            class_average_activations_per_layer[class_index][layer_index] = np.squeeze((tf.divide(layer[0], np.sum(y_train == class_index))).numpy())

    print("Averaging gradient classes..")
    for class_index, class_gradients in enumerate(class_average_gradients_per_layer):
        class_average_gradients_per_layer[class_index] = [np.squeeze(tf.divide([class_gradients[i]], np.sum(y_train == class_index)).numpy()) for i in range(len(class_gradients))]

    print("Averaging correct prediction activation classes..")
    for class_index, class_activations in enumerate(class_correct_average_activations_per_layer):
        for layer_index, layer in enumerate(class_activations):
            if class_correct_amounts[class_index] >= 1:
                class_correct_average_activations_per_layer[class_index][layer_index] = np.squeeze((tf.divide(layer[0], class_correct_amounts[class_index])).numpy())

    print("Averaging incorrect prediction activation classes..")
    for class_index, class_activations in enumerate(class_incorrect_average_activations_per_layer):
        for layer_index, layer in enumerate(class_activations):
            if class_incorrect_amounts[class_index] >= 1:
                class_incorrect_average_activations_per_layer[class_index][layer_index] = np.squeeze((tf.divide(layer[0], class_incorrect_amounts[class_index])).numpy())

    print("Computing overall signal averages..")
    average_signals_per_layer = []
    num_layers = len(class_average_signals_per_layer[0])
    for layer_index in range(num_layers):
        summed_activations = tf.zeros_like(class_average_signals_per_layer[0][layer_index])

        for class_activations in class_average_signals_per_layer:
            summed_activations = tf.add(summed_activations, class_activations[layer_index])

        average_signals_per_layer.append(summed_activations)

    for layer_index in range(num_layers):
        average_signals_per_layer[layer_index] = np.squeeze(tf.divide(average_signals_per_layer[layer_index], x_train.shape[0]).numpy())

    print("Averaging signal classes..")
    for class_index, class_signals in enumerate(class_average_signals_per_layer):
        for layer_index, layer in enumerate(class_signals):
            class_average_signals_per_layer[class_index][layer_index] = np.squeeze(tf.divide(layer[0], np.sum(y_train == class_index)).numpy())

    print("Averaging correct prediction signal classes..")
    for class_index, class_activations in enumerate(class_correct_average_signals_per_layer):
        for layer_index, layer in enumerate(class_activations):
            if class_correct_amounts[class_index] >= 1:
                class_correct_average_signals_per_layer[class_index][layer_index] = np.squeeze((tf.divide(layer[0], class_correct_amounts[class_index])).numpy())
    print("Averaging incorrect prediction signal classes..")
    for class_index, class_activations in enumerate(class_incorrect_average_signals_per_layer):
        for layer_index, layer in enumerate(class_activations):
            if class_incorrect_amounts[class_index] >= 1:
                class_incorrect_average_signals_per_layer[class_index][layer_index] = np.squeeze((tf.divide(layer[0], class_incorrect_amounts[class_index])).numpy())

    print("Saving results..")
    file_path = 'saved_precalculations/' + model_name + '/average_activations.pickle'
    with open(file_path, 'wb') as handle:
        pickle.dump(average_activations_per_layer, handle, protocol=pickle.HIGHEST_PROTOCOL)

    file_path = 'saved_precalculations/' + model_name + '/average_signals.pickle'
    with open(file_path, 'wb') as handle:
        pickle.dump(average_signals_per_layer, handle, protocol=pickle.HIGHEST_PROTOCOL)

    file_path = 'saved_precalculations/' + model_name + '/class_average_activations.pickle'
    with open(file_path, 'wb') as handle:
        pickle.dump(class_average_activations_per_layer, handle, protocol=pickle.HIGHEST_PROTOCOL)

    file_path = 'saved_precalculations/' + model_name + '/class_average_signals.pickle'
    with open(file_path, 'wb') as handle:
        pickle.dump(class_average_signals_per_layer, handle, protocol=pickle.HIGHEST_PROTOCOL)

    file_path = 'saved_precalculations/' + model_name + '/class_average_gradients.pickle'
    with open(file_path, 'wb') as handle:
        pickle.dump(class_average_gradients_per_layer, handle, protocol=pickle.HIGHEST_PROTOCOL)

    file_path = 'saved_precalculations/' + model_name + '/class_correct_average_activations.pickle'
    with open(file_path, 'wb') as handle:
        pickle.dump(class_correct_average_activations_per_layer, handle, protocol=pickle.HIGHEST_PROTOCOL)

    file_path = 'saved_precalculations/' + model_name + '/class_incorrect_average_activations.pickle'
    with open(file_path, 'wb') as handle:
        pickle.dump(class_incorrect_average_activations_per_layer, handle, protocol=pickle.HIGHEST_PROTOCOL)

    file_path = 'saved_precalculations/' + model_name + '/class_correct_average_signals.pickle'
    with open(file_path, 'wb') as handle:
        pickle.dump(class_correct_average_signals_per_layer, handle, protocol=pickle.HIGHEST_PROTOCOL)

    file_path = 'saved_precalculations/' + model_name + '/class_incorrect_average_signals.pickle'
    with open(file_path, 'wb') as handle:
        pickle.dump(class_incorrect_average_signals_per_layer, handle, protocol=pickle.HIGHEST_PROTOCOL)

    return